source: https://towardsdatascience.com/bring-your-jupyter-notebook-to-life-with-interactive-widgets-bc12e03f0916

In [1]:
import ipywidgets as widgets
import pandas as pd
import numpy as np

In [2]:
btn = widgets.Button(description='Medium')
display(btn)

def btn_eventhandler(obj):
    print('Hello from the {} button!'.format(obj.description))
    
btn.on_click(btn_eventhandler)

Button(description='Medium', style=ButtonStyle())

Hello from the Medium button!


In [3]:
url = "https://data.london.gov.uk/download/number-international-visitors-london/b1e0f953-4c8a-4b45-95f5-e0d143d5641e/international-visitors-london-raw.csv"
df_london = pd.read_csv(url)

df_london.sample(5)

,year,quarter,market,dur_stay,mode,purpose,area,visits,spend,nights,sample
42730,2014,Q3,France,1-3 nights,Sea,Business,LONDON,3.775468,0.066791,3.952741,3
34878,2012,Q2,Sweden,4-7 nights,Air,Business,LONDON,3.774597,2.612408,16.410034,9
45297,2015,Q2,Germany,4-7 nights,Tunnel,Holiday,LONDON,4.621740,1.723666,31.539483,3
55606,2018,Q2,Czech Republic,1-3 nights,Air,Miscellaneous,LONDON,0.451077,0.099237,0.451077,1
21508,2008,Q2,Other Eastern Europe,4-7 nights,Air,Business,LONDON,4.357961,3.357664,20.831400,6


In [4]:
ALL = 'ALL'

def unique_sorted_values_plus_ALL(array):
    unique = array.unique().tolist()
    unique.sort()
    unique.insert(0, ALL)
    return unique

In [5]:
dropdown_year = widgets.Dropdown(options=unique_sorted_values_plus_ALL(df_london.year))
output_year = widgets.Output()

def dropdown_year_eventhandler(change):
    output_year.clear_output()
    
    with output_year:
        if (change.new == ALL):
            display(df_london)
        else:
            display(df_london[df_london.year == change.new])

dropdown_year.observe(dropdown_year_eventhandler, names='value')

display(dropdown_year)

Dropdown(options=('ALL', 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2…

In [6]:
display(output_year)

Output()

In [12]:
output = widgets.Output()

# create 2 dropdown widgets
dropdown_year = widgets.Dropdown(options=unique_sorted_values_plus_ALL(df_london.year))
dropdown_purpose = widgets.Dropdown(options=unique_sorted_values_plus_ALL(df_london.purpose))

# generic filtering on year and purpose
def common_filtering(year, purpose):
    output.clear_output()
    
    if (year == ALL) & (purpose == ALL):
        common_filter = df_london
    elif (year == ALL):
        common_filter = df_london[df_london.purpose == purpose]
    elif (purpose == ALL):
        common_filter = df_london[df_london.year == year]
    else:
        common_filter = df_london[(df_london.year == year) & (df_london.purpose == purpose)]
    
    with output:
        display(common_filter)

# set event handler to use common_filtering        
def dropdown_year_eventhandler(change):
    common_filtering(change.new, dropdown_purpose.value)
    
def dropdown_purpose_eventhandler(change):
    common_filtering(dropdown_year.value, change.new)

# binding the handler
dropdown_year.observe(dropdown_year_eventhandler, names='value')
dropdown_purpose.observe(dropdown_purpose_eventhandler, names='value')

display(dropdown_year)
display(dropdown_purpose)

Dropdown(options=('ALL', 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2…

Dropdown(options=('ALL', 'Business', 'Holiday', 'Miscellaneous', 'Study', 'VFR'), value='ALL')

In [14]:
display(output)

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': '      year quarter             market …

In [16]:
bounded_num = widgets.BoundedFloatText(min=0, max=100000, value=5, step=1)

def colour_ge_value(value, comparison):
    if value >= comparison:
        return 'color: red'
    else:
        return 'color: black'

In [17]:
output = widgets.Output()

# create 2 dropdown widgets
dropdown_year = widgets.Dropdown(options=unique_sorted_values_plus_ALL(df_london.year))
dropdown_purpose = widgets.Dropdown(options=unique_sorted_values_plus_ALL(df_london.purpose))

# generic filtering on year and purpose
def common_filtering(year, purpose, num):
    output.clear_output()
    
    if (year == ALL) & (purpose == ALL):
        common_filter = df_london
    elif (year == ALL):
        common_filter = df_london[df_london.purpose == purpose]
    elif (purpose == ALL):
        common_filter = df_london[df_london.year == year]
    else:
        common_filter = df_london[(df_london.year == year) & (df_london.purpose == purpose)]
    
    with output:
        display(
            common_filter.style.applymap(
                lambda x: colour_ge_value(x, num),
                subset=['visits','spend', 'nights']
            )
       )

# set event handler to use common_filtering        
def dropdown_year_eventhandler(change):
    common_filtering(change.new, dropdown_purpose.value, bounded_num.value)
    
def dropdown_purpose_eventhandler(change):
    common_filtering(dropdown_year.value, change.new, bounded_num.value)
  
def bounded_num_eventhandler(change):
    common_filtering(dropdown_year.value, dropdown_purpose.value, change.new)


# binding the handler
dropdown_year.observe(dropdown_year_eventhandler, names='value')
dropdown_purpose.observe(dropdown_purpose_eventhandler, names='value')
bounded_num.observe(bounded_num_eventhandler, names='value')

display(dropdown_year)
display(dropdown_purpose)
display(bounded_num)

Dropdown(options=('ALL', 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2…

Dropdown(options=('ALL', 'Business', 'Holiday', 'Miscellaneous', 'Study', 'VFR'), value='ALL')

BoundedFloatText(value=5.0, max=100000.0, step=1.0)

In [18]:
display(output)

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': '<pandas.io.formats.style.Styler at 0x7…

In [23]:
import seaborn as sns
import matplotlib.pyplot as plt

output = widgets.Output()
plot_output = widgets.Output()

# create 2 dropdown widgets
dropdown_year = widgets.Dropdown(options=unique_sorted_values_plus_ALL(df_london.year))
dropdown_purpose = widgets.Dropdown(options=unique_sorted_values_plus_ALL(df_london.purpose))

# generic filtering on year and purpose
def common_filtering(year, purpose, num):
    output.clear_output()
    plot_output.clear_output()
    
    if (year == ALL) & (purpose == ALL):
        common_filter = df_london
    elif (year == ALL):
        common_filter = df_london[df_london.purpose == purpose]
    elif (purpose == ALL):
        common_filter = df_london[df_london.year == year]
    else:
        common_filter = df_london[(df_london.year == year) & (df_london.purpose == purpose)]
    
    with output:
        display(common_filter.style.applymap(lambda x: colour_ge_value(x, num),subset=['visits','spend', 'nights']))
        
    with plot_output:
        sns.kdeplot(common_filter['visits'], shade=True)
        plt.show()

# set event handler to use common_filtering        
def dropdown_year_eventhandler(change):
    common_filtering(change.new, dropdown_purpose.value, bounded_num.value)
    
def dropdown_purpose_eventhandler(change):
    common_filtering(dropdown_year.value, change.new, bounded_num.value)
  
def bounded_num_eventhandler(change):
    common_filtering(dropdown_year.value, dropdown_purpose.value, change.new)


# binding the handler
dropdown_year.observe(dropdown_year_eventhandler, names='value')
dropdown_purpose.observe(dropdown_purpose_eventhandler, names='value')
bounded_num.observe(bounded_num_eventhandler, names='value')

display(dropdown_year)
display(dropdown_purpose)
display(bounded_num)

Dropdown(options=('ALL', 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2…

Dropdown(options=('ALL', 'Business', 'Holiday', 'Miscellaneous', 'Study', 'VFR'), value='ALL')

BoundedFloatText(value=2.0, max=100000.0, step=1.0)

In [24]:
display(output)
display(plot_output)

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': '<pandas.io.formats.style.Styler at 0x7…

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': '<Figure size 432x288 with 1 Axes>', 'i…

In [31]:
item_layout = widgets.Layout(margin='0 0 20px 0')

input_widgets = widgets.HBox([dropdown_year, dropdown_purpose, bounded_num], layout=item_layout)
display(input_widgets)

In [32]:
tab = widgets.Tab([output, plot_output], layout=item_layout)
tab.set_title(0, 'Dataset Exploration')
tab.set_title(1, 'KDE Plot')
display(tab)

In [33]:
dashboard = widgets.VBox([input_widgets, tab])
display(dashboard)

In [56]:
import maya

debug_view = widgets.Output(layout={'border': '1px solid black'})

@debug_view.capture(clear_output=True)
def bad_callback(event):
    now = maya.now().rfc2822()
    
    print(f'This is about to explode: {now}')
#     return 1.0 / 0.0

btn = widgets.Button(
    description='click me to raise an exception',
    layout={'width': '300px'}
)

btn.on_click(bad_callback)
display(btn)

debug_view

Button(description='click me to raise an exception', layout=Layout(width='300px'), style=ButtonStyle())

Output(layout=Layout(border='1px solid black'))

In [55]:
# declare the output
out = widgets.Output(layout={'border': '1px solid black'})
# declate the interactive element
btn = widgets.Button(description='Medium')

# declare the function to apply
def f():
    return maya.now().rfc2822() 

# declare the event handler
def btn_eventhandler(obj):
    out.description = f'Hellow from the {obj.description}, {f()}'
#     out.clear_output()
#     with out:
#         print(f'Hello from the {obj.description} button!')

# call the observe() method with event handler and value
btn.observe(btn_eventhandler, names='description')

display(btn)

display(out)

Button(description='Medium', style=ButtonStyle())

Output(layout=Layout(border='1px solid black'))

In [41]:
btn.description

'Medium'

In [51]:
a = widgets.IntSlider(description='a')
b = widgets.IntSlider(description='b')
c = widgets.IntSlider(description='c')
def f(a, b, c):
    print('{}*{}*{}={}'.format(a, b, c, a*b*c))

out = widgets.interactive_output(f, {'a': a, 'b': b, 'c': c})

widgets.HBox([widgets.VBox([a, b, c]), out])